In [1]:
'''
Modifying the era5 data to be compatible with the TempestExtremes tracking software

'''


# import packages
%pylab inline

import xarray as xr
import cftime
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
# redo time for HIRAM

ds = xr.open_mfdataset("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/model_test/20190101.atmos_4xdaily.nc").load()
ds['time'].encoding['units'] = 'hours since 0001-01-01 00:00:00'
ds['time'].encoding['calendar'] = 'gregorian'

In [5]:
ds.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/model_test/20190101.atmos_4xdaily.nc")
    

### Renaming time variable from 'valid_time' to 'time' and changing time units from seconds to hours

In [3]:
# SLP files

dloc = '/tigress/wenchang/data/era5/raw/4xdaily/slp'
year_list = ['2021', '2022', '2023', '2024']

for year in year_list:
    ds = xr.open_mfdataset("{}/era5.mean_sea_level_pressure.{}.nc".format(dloc, year)).load()
    ds = ds.rename({'valid_time': 'time'})
    ds['time'].encoding['units'] = 'hours since 1970-01-01 00:00:00'
    ds['time'].encoding['calendar'] = 'gregorian'
    encoding = {'msl': {'zlib': False}}
    ds.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.mean_sea_level_pressure.{}.nc".format(year), encoding=encoding)
    print(year)


#slp.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.mean_sea_level_pressure.{}.nc".format(year)

2021
2022
2023
2024


In [6]:
# u10 files

dloc = '/tigress/wenchang/data/era5/raw/4xdaily/u10'
year_list = ['2021', '2022', '2023', '2024']

for year in year_list:
    u10 = xr.open_mfdataset("{}/era5.10m_u_component_of_wind.{}.nc".format(dloc, year)).load()
    u10 = u10.rename({'valid_time': 'time'})
    u10['time'].encoding['units'] = 'hours since 1970-01-01 00:00:00'
    u10['time'].encoding['calendar'] = 'gregorian'
    encoding = {'u10': {'zlib': False}}
    u10.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.10m_u_component_of_wind.{}.nc".format(year), encoding=encoding)
    print(year)


2020


In [7]:
# v10 files

dloc = '/tigress/wenchang/data/era5/raw/4xdaily/v10'
year_list = ['2021', '2022', '2023', '2024']

for year in year_list:
    v10 = xr.open_mfdataset("{}/era5.10m_v_component_of_wind.{}.nc".format(dloc, year)).load()
    v10 = v10.rename({'valid_time': 'time'})
    v10['time'].encoding['units'] = 'hours since 1970-01-01 00:00:00'
    v10['time'].encoding['calendar'] = 'gregorian'
    encoding = {'v10': {'zlib': False}}
    v10.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.10m_v_component_of_wind.{}.nc".format(year), encoding=encoding)
    print(year)


2020


### Doing the following for the geopotential height:

1. concatenate the 2 levels across the variable 'level'
2. transpose the concatenated data such that the coordinates are (time, level, latitude, longitude)
3. make sure level has units of hPa
4. make sure level is float32 not int64
5. change time variable from 'valid_time' to 'time'
6. change units of time from seconds to hours



In [4]:
# geopotential height files

dloc500 = '/tigress/wenchang/data/era5/raw/4xdaily/h500'
dloc300 = '/tigress/wenchang/data/era5/raw/4xdaily/h300'
year_list = ['2021', '2022']

for year in year_list:
    h500 = xr.open_mfdataset("{}/era5.geopotential.500.{}.nc".format(dloc500, year)).load()
    h300 = xr.open_mfdataset("{}/era5.geopotential.300.{}.nc".format(dloc300, year)).load()

    # rename variable
    h500 = h500.rename({'pressure_level': 'level'})
    h300 = h300.rename({'pressure_level': 'level'})
    
    # Step 2: Ensure 'level' has correct units and dtype (float32, hPa)
    h500['level'].attrs['units'] = 'hPa'
    h300['level'].attrs['units'] = 'hPa'
    h500['level'] = h500['level'].astype(np.float32)
    h300['level'] = h300['level'].astype(np.float32)
    
    # concatenate
    z = xr.concat([h500, h300], dim='level')
    
    # rename time and ensure proper units
    z = z.rename({'valid_time': 'time'})
    z['time'].encoding['units'] = 'hours since 1970-01-01 00:00:00'
    z['time'].encoding['calendar'] = 'gregorian'
    encoding = {'z': {'zlib': False}}
    
    #save to file
    z.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.geopotential.{}.nc".format(year), encoding=encoding)
    print(year)


2021
2022


In [5]:


# geopotential height files

dloc500 = '/tigress/wenchang/data/era5/raw/4xdaily/h500'
dloc300 = '/tigress/wenchang/data/era5/raw/4xdaily/h300'
year_list = ['2004', '2005']

for year in year_list:
    h500 = xr.open_mfdataset("{}/era5.geopotential.500.{}.nc".format(dloc500, year)).load()
    h300 = xr.open_mfdataset("{}/era5.geopotential.300.{}.nc".format(dloc300, year)).load()
    
    z = xr.concat([h500, h300], dim=xr.DataArray([500, 300], dims='level', name='level'))
    print('step 1')

    # units
    z['level'].attrs['units'] = 'hPa'
    z['level'] = z['level'].astype(np.float32)

    # transpose for right dimensions
    z = z.transpose('time', 'level', 'latitude', 'longitude')
    
    
    #save to file
    z.to_netcdf("/home/el2358/GEOCLIM/el2358/projects/tc_tracker/era5_data_input/era5.geopotential.{}.nc".format(year))
    print(year)


step 1
2004
step 1
2005


In [3]:
z

<xarray.Dataset> Size: 24GB
Dimensions:    (level: 2, time: 1460, latitude: 721, longitude: 1440)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 12kB 2005-01-01 ... 2005-12-31T18:00:00
  * level      (level) int64 16B 500 300
Data variables:
    z          (level, time, latitude, longitude) float64 24GB 5.012e+04 ... ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-05-04 03:09:04 GMT by grib_to_netcdf-2.10.0: /opt/ecmw...